In [1]:
#Load keys
import os
from dotenv import load_dotenv
load_dotenv()

# Load environment variables from .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
##langsmith tracking
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
langsmith_project = os.getenv('LANGSMITH_PROJECT')
if langsmith_project is not None:
	os.environ['LANGSMITH_PROJECT'] = langsmith_project
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [17]:
#Data Ingestion from the website

from langchain_community.document_loaders import WebBaseLoader
Loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
Loader

In [18]:
docs = Loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommende

In [19]:
docs[0].page_content

'\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefore diving into this content, it might be helpful to read the following:\nData Retention Conceptual Docs\nUsage Limiting Conceptual Docs\n\nnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you ar

In [20]:
#Splitting the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)                                      

In [21]:
#Converting chunks into embeddings(vectors)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [22]:
#Storing the embeddings in vector database
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents,embeddings)



In [23]:
#Querying the vector database
query = "LangSmith has two usage limits: total traces and extended retention traces"

#Retrieval
result = vectorstore.similarity_search(query)
result[0].page_content

'Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estimate:\n\nLets start by setting limits on our production usage, since that is where the majority of spend comes from.\nSetting a good total traces limit\u200b\nPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your assumptions before setting a lim

In [28]:
#Defining the LLM model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")

In [29]:
#Retrieveal chain 

from langchain.chains.combine_documents import create_stuff_documents_chain

#Importing chat prompt template
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

#document chain
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000221E853C520>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000221E853E5C0>, root_client=<openai.OpenAI object at 0x00000221E853C490>, root_async_client=<openai.AsyncOpenAI object at 0x00000221E853C430>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'

In [33]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and extended traces retention traces. The total traces limit is the maximum number of traces that can be stored in LangSmith at any given time. The extended retention traces limit is the maximum number of traces that can be stored for an extended period of time, beyond the default retention period.")]
})

'LangSmith has two usage limits: the total traces limit and the extended retention traces limit. The total traces limit is the maximum number of traces that can be stored in LangSmith at any given time. The extended retention traces limit is the maximum number of traces that can be stored for an extended period of time, beyond the default retention period.'

In [38]:
#Retriever

retriever = vectorstore.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

#docuemnt chain is responsible for giviing the right contextual information

In [39]:
#Getting the response from llm
response = retrieval_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended retention traces"
})

response

{'input': 'LangSmith has two usage limits: total traces and extended retention traces',
 'context': [Document(id='5c942055-7941-4c6a-962b-7639517ba713', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you

In [40]:
response['answer']

'To set usage limits for LangSmith, you would navigate to the "Settings" menu, then select "Usage and Billing," followed by "Usage configuration." At the bottom of the page, you\'ll find a table where you can set usage limits per workspace, specifically for "total traces" and "extended retention traces." This configuration will help prevent future overspending by allowing granular control of the spend per workspace.'